# Import

In [1]:
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm

from openai import OpenAI

from appworld import AppWorld, load_task_ids

# Env Setup

In [2]:
env_path = os.path.join("config", ".env")

load_dotenv(env_path)

True

# OpenAI Response API

## Simple response request

In [ ]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
response = llm_client.responses.create(
    model="gpt-4o",
    temperature=0.0,
    input=[
        {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
        {'role' : 'user', 'content' : '안녕하세요 당신은 누구인가요?'}
    ]
)

# Output Parsing
if hasattr(response, "output"):
    for output in response.output:
        if hasattr(output, "content"):
            for content in output.content:
                if hasattr(content, "text"):
                    print(content.text)

안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?


In [11]:
print(response)

Response(id='resp_000b7636b507e0e900694382c64b4c81939aa2cc96dedb10a1', created_at=1766032070.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_000b7636b507e0e900694382c700908193b54878f8b067e23c', content=[ResponseOutputText(annotations=[], text='안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='di

## Tool Call


| Field | Description |
| :---: | :---: |
| type	| This should always be function |
| name	| The function's name (e.g. get_weather) |
| description	| Details on when and how to use the function |
| parameters	| JSON schema defining the function's input arguments |
| strict	| Whether to enforce strict mode for the function call |

In [4]:
TOOLS = [
    {
        "type": "function",
        "name": "get_weather",
        "description": """
        이 도구는 특정 도시의 현재 날씨를 조회하기 위해 사용합니다.
        
        이 도구를 활용하기 위한 입력 인자는 다음과 같습니다.
        <입력 인자>
        location: 도시 이름
        units: 온도 단위 (celsius, fahrenheit, None)
        </입력 인자>

        이 도구를 사용하기 위한 JSON Schema 예시는 다음과 같습니다.
        <JSON Schema>
        {
            "location": "Seoul, KR",
            "units": "celsius"
        }
        </JSON Schema>
        """,
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "예: Seoul, KR"},
                "units": {"type": ["string", "null"], "enum": ["celsius", "fahrenheit", None]},
            },
            "required": ["location", "units"],
            "additionalProperties": False,
        },
    },
]

In [3]:
def get_weather(location, units=None):
    return f"Weather in {location} is 10 {units}."

In [25]:
from typing import Literal

def tmp(a : int, b:Literal["1", "2"]):
    return a, b

In [23]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

trajectory = [
    {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
    {'role' : 'user', 'content' : '현재 서울의 날씨는 어떤가요?'}
]

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

trajectory += response.output

for output in response.output:
    if output.type == "function_call":
        print("✅ Tool Call Maded!")
        if output.name == "get_weather":
            # execute tool function
            tool_result = get_weather(**json.loads(output.arguments))
            print("✅ Tool Function Executed!")
            print(f"     📍 Result : {tool_result}")

            # add tool result to trajectory
            trajectory.append({
                'type' : 'function_call_output',
                'call_id' : output.call_id,
                'output' : json.dumps({
                    "weather_info" : tool_result
                })
            })           

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

for output in response.output:
    if output.type == "message":
        for content in output.content:
            if content.type == "output_text":
                print(content.text)

✅ Tool Call Maded!
✅ Tool Function Executed!
     📍 Result : Weather in Seoul, KR is 10 celsius.
서울의 현재 날씨는 섭씨 10도입니다.


In [24]:
trajectory

[{'role': 'system', 'content': '당신의 이름은 ReflACE Agent 입니다.'},
 {'role': 'user', 'content': '현재 서울의 날씨는 어떤가요?'},
 ResponseFunctionToolCall(arguments='{"location":"Seoul, KR","units":"celsius"}', call_id='call_gtDiWI8sjnkqFxWcZddRODkT', name='get_weather', type='function_call', id='fc_02d448c8cd97ffb900694387d9003c81908123434c62c606c3', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_gtDiWI8sjnkqFxWcZddRODkT',
  'output': '{"weather_info": "Weather in Seoul, KR is 10 celsius."}'}]

## AppWorld

In [3]:
TOOLS = [
    {
        "type" : "function",
        "name" : "appworld_execute",
        "description" : """
        Execute Python code inside the AppWorld IPython-like execution shell. 
        The shell is stateful across calls. 
        Use `apis.{app}.{api}(...)` or `requester.{method}(...)` inside the code. 
        When you finish the task, call `apis.supervisor.complete_task()` 
        (optionally with answer=...).
        """,
        "parameters" : {
            "type" : "object",
            "properties" : {
                "code" : {"type" : "string", "description" : "Python code to run in AppWorld shell"}
            },
            "required" : ["code"],
            "additionalProperties" : False
        },
        "strict" : True
    }
]

In [ ]:
# load task ids
task_ids = load_task_ids("dev")

# select task id
task_id = task_ids[0]
print(f"📍 Current task ID: {task_id}")

# initialize task environment
env = AppWorld(task_id=task_id, experiment_name="sample")
print(f"✅ Task Environment Initialized!")

# get task instruction
instruction = env.task.instruction
print(f"🎯 Task Instruction: {instruction}")

# make system prompt
with open(os.path.join('src','prompt','react','system_prompt.txt'), 'r', encoding='utf-8') as f:
    system_prompt = f.read()

trajectory = [
    {'role' : 'user', 'content' : instruction}
]

# initialize llm core client
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

complete_flag = False
max_step = 20
for step in range(max_step):
    response = openai_client.responses.create(
        model='gpt-4o',
        instructions=system_prompt,
        input=trajectory,
        tools=TOOLS,
        temperature=0.0
    )

    trajectory += response.output

    tool_calls = [item for item in response.output if item.type == 'function_call']

    if not tool_calls:
        print("✅ Agent has finished the task.")
        print(f"📍Result :\n{response.output[0].content[0].text}")
        break

    for tool_call in tool_calls:
        if tool_call.name != 'appworld_execute':
            raise ValueError(f"Unknown tool call: {tool_call.name}")
        
        code = json.loads(tool_call.arguments)['code']

        obs = env.execute(code)

        trajectory.append(
            {
                'type' : 'function_call_output',
                'call_id' : tool_call.call_id,
                'output' : obs
            }
        )

📍 Current task ID: 50e1ac9_1
✅ Task Environment Initialized!
🎯 Task Instruction: Give me a comma-separated list of top 4 most played r&b song titles from across my Spotify song, album and playlist libraries.
✅ Agent has finished the task.
📍Result :
The top 4 most played R&B songs are: Haunted Memories, Shadows of the Past, Beneath the Veil of Illusion, When Fate Becomes a Foe.


In [31]:
trajectory

[{'role': 'user',
  'content': 'Give me a comma-separated list of top 4 most played r&b song titles from across my Spotify song, album and playlist libraries.'},
 ResponseFunctionToolCall(arguments='{"code":"print(apis.api_docs.show_api_descriptions(app_name=\'spotify\'))"}', call_id='call_IvfUGJ0hXrDgPXSe8CDtjYDY', name='appworld_execute', type='function_call', id='fc_081dc840b5d20817006944120d34f8819c93dea41a6263dda8', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_IvfUGJ0hXrDgPXSe8CDtjYDY',
  'output': '[\n {\n  "name": "show_account",\n  "description": "Show your account information. Unlike show_profile, this includes private information."\n },\n {\n  "name": "signup",\n  "description": "Sign up to create account."\n },\n {\n  "name": "delete_account",\n  "description": "Delete your account."\n },\n {\n  "name": "update_account_name",\n  "description": "Update your first or last name in the account profile."\n },\n {\n  "name": "login",\n  "description": 

In [32]:
evaluation = env.evaluate()

evaluation.report()

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 1

Num Failed Tests : 1

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

────────────────────────────────────────────────────── Fails ──────────────────────────────────────────────────────

>> Failed Requirement

assert answers match.

```python                                                                                                          
with test(                                                                                                         
    """                                                                                                            
    assert answers match.                                                                                          
    """                                                                                                            
):                                                                                                                 
    ground_truth_song_titles = ground_truth_answer.split(",")                                                      
    predicted_song_titles = predicted_answer.split(",")                                                            
    test.case(                                                                                                     
```                                                                                                                
----------                                                                                                         
AssertionError:                                                                                                    
['crimson veil', 'fire and ice', 'haunted memories', 'mysteries of the silent sea']                                
==                                                                                                                 
['beneath the veil of illusion', 'haunted memories', 'shadows of the past', 'when fate becomes a foe']             
                                                                                                                   
In left but not right:                                                                                             
['crimson veil', 'fire and ice', 'mysteries of the silent sea']                                                    
                                                                                                                   
In right but not left:                                                                                             
['beneath the veil of illusion', 'shadows of the past', 'when fate becomes a foe']                                 
                                                                                                                   
Original values:                                                                                                   
['Mysteries of the Silent Sea', 'Crimson Veil', 'Haunted Memories', 'Fire and Ice']                                
==                                                                                                                 
['Haunted Memories', ' Shadows of the Past', ' Beneath the Veil of Illusion', ' When Fate Becomes a Foe']          

''